In [1]:
import sys
import numpy as np
import torch
import learn2learn as l2l
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms

from PIL.Image import LANCZOS
from csv import writer

from PIL import Image
import pandas as pd

In [2]:
torch.cuda.is_available()

False

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.7.1002181843/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2022.7.1002181843/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python39.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
# from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder
# from src.zoo.trident_utils import setup, inner_adapt_trident, loss, accuracy
# from data.loaders import CUBirds200
from src.zoo.trident_utils import inner_adapt_trident, setup


In [5]:
device = 'cpu'
n_ways = 5
k_shots = 1
q_shots = 10
order = False
inner_lr = 0.001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 5
meta_batch_size = 10
download = False

In [6]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'tiered'
        self.root = '../../dataset/mini_imagenet/'
        self.task_adapt = True
        self.task_adapt_fn = 'tafe'
        self.n_ways = 5
        self.k_shots = 1
        self.q_shots = 10
        self.zl = 64
        self.zs = 64
        self.reconstr = 'std'
        self.device = 'cpu'
        self.wn_channels = 32
        self.wm_channels = 64

args = Args(100, False, 0.01, 1, 1)

In [7]:
m = nn.LeakyReLU(0.2)
input = torch.randn(2)
output = m(input)
output

tensor([1.2311, 1.3346])

In [8]:
m = nn.LeakyReLU(0.2)
#m = m(0.2)
input = torch.randn(2)
output = m(input)
output

tensor([-0.0233, -0.3031])

In [11]:
model = CEncoder(3, 32, args=args)
input = ttask[0]
output = model(input)
output

tensor([[-0.2709,  0.7311,  0.5055,  ...,  0.5440,  0.1957,  0.4542],
        [ 1.3763,  0.2670,  0.2804,  ...,  0.1724,  0.2297, -0.0047],
        [ 0.0390,  0.5507,  0.3055,  ...,  1.2002,  0.9567,  0.0426],
        ...,
        [ 1.1826,  0.9422, -0.0968,  ...,  1.6351,  0.8731,  0.7799],
        [ 0.6207, -0.0732,  0.0770,  ...,  1.9577,  1.0335,  1.7338],
        [ 0.8392, -0.0134,  0.6926,  ...,  0.3367,  0.7953,  0.0594]],
       grad_fn=<ReshapeAliasBackward0>)

In [16]:
learner = torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500.pt')

In [17]:
learner.args

Namespace(cnfg='/home/nfs/anujsingh/meta_lrng/files/temp/learning_to_meta-learn/configs/tiered-5,1/exp5-tafe.json', dataset='tiered', root='../../dataset', model_path='/home/nfs/anujsingh/meta_lrng/files/temp/learning_to_meta-learn/logs/DELPO_tiered_5-way_1-shot_15-queries/models-5-tafe', pretrained=None, n_ways=5, k_shots=1, q_shots=10, inner_adapt_steps_train=5, inner_adapt_steps_test=5, inner_lr=0.0015, meta_lr=0.00015, meta_batch_size=20, iterations=100000, wt_ce=150, klwt=False, rec_wt=0.01, beta_l=1, beta_s=1, task_adapt=True, task_adapt_fn='tafe', alpha=0.1, experiment='exp5-tafe', order=False, device='cuda', download=None, resume='No', iter_resume=0)

In [18]:
learner.modules

<bound method Module.modules of MAML(
  (module): CCVAE(
    (encoder): CEncoder(
      (net): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (12): Conv2d(32, 32, kernel_size=(3, 3), str

In [19]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-2.5233e-01, -1.3083e-02,  1.7343e-01],
                        [-6.0568e-02, -5.9060e-02,  1.0598e-01],
                        [-2.7154e-01, -1.7081e-02,  4.0474e-02]],
              
                       [[-3.5261e-01, -1.5088e-02,  3.7616e-02],
                        [-2.7602e-01,  4.7716e-02, -1.3594e-01],
                        [-5.7986e-02,  5.9228e-02, -1.3645e-01]],
              
                       [[-3.5785e-03,  4.1307e-02, -1.2321e-01],
                        [-1.9926e-01,  9.2488e-02,  7.2494e-02],
                        [-2.4462e-01,  4.5338e-02,  2.4103e-02]]],
              
              
                      [[[ 1.6439e-01, -3.5624e-02,  1.8537e-01],
                        [-1.6574e-01, -3.6141e-02,  2.3362e-01],
                        [ 1.2839e-01,  1.2165e-01,  8.9327e-02]],
              
                       [[ 7.7133e-02,  5.7750e-02,  2.0098e-01],
                        [ 1.04

In [20]:
torch.save(learner.state_dict(), '../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt')

In [8]:
learner = CCVAE(in_channels=3, base_channels=32,
                        n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
#opt = optim.Adam(learner.parameters(), 0.00015)

In [9]:
train_tasks, valid_tasks, _, learner = setup(
    args.dataset, args.root, args.n_ways, args.k_shots, args.q_shots, order, inner_lr, args.device, download=download, task_adapt=args.task_adapt, args=args)
opt = optim.Adam(learner.parameters(), meta_lr)
reconst_loss = nn.MSELoss(reduction='none')


In [10]:
ttask = train_tasks.sample()
model = learner.clone()

In [9]:
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

In [10]:
evaluation_loss

{'elbo': tensor(12.8777, grad_fn=<AddBackward0>),
 'label_kl': tensor(1.6585, grad_fn=<MeanBackward0>),
 'style_kl': tensor(9.3425, grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(0.1450, grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.7316, grad_fn=<NllLossBackward0>)}

In [11]:
evaluation_loss['elbo'].backward()

In [12]:
torch.save(learner.state_dict(), './temp.pt')

In [9]:
learner1 = CCVAE(in_channels=3, base_channels=32, n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
learner1 = l2l.algorithms.MAML(learner1, first_order=order, lr=inner_lr, allow_nograd=False)
learner1.load_state_dict(torch.load('../logs/DELPO_miniimagenet_5-way_5-shot_10-queries/og/model_28500.pt'))

<All keys matched successfully>

In [10]:
learner1.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[ 2.3819e-01, -1.4950e-01,  9.3198e-02],
                        [-3.1704e-02, -5.7982e-02, -2.0686e-01],
                        [-1.1069e-01, -1.6322e-01, -1.4552e-01]],
              
                       [[ 1.6611e-01, -1.5929e-02,  1.2639e-01],
                        [-1.8461e-01, -4.3771e-02, -1.6856e-01],
                        [ 1.2173e-01, -1.1646e-01,  1.2756e-01]],
              
                       [[ 1.2097e-01,  2.2649e-02,  6.5315e-02],
                        [-8.8836e-02,  1.5429e-01,  7.6307e-02],
                        [ 8.4700e-02, -8.8268e-02, -1.8083e-01]]],
              
              
                      [[[-1.2549e-01,  4.3194e-02,  2.6716e-03],
                        [ 1.0144e-01, -3.2592e-02,  2.8972e-02],
                        [ 8.9148e-02, -8.9075e-02, -2.0863e-01]],
              
                       [[-5.7803e-03, -2.1918e-02, -2.6987e-02],
                        [-1.46

In [11]:
learner = CCVAE(in_channels=3, base_channels=32, n_ways=args.n_ways, task_adapt=True, args=args, latent_dim_l=args.zl, latent_dim_s=args.zs)
learner = l2l.algorithms.MAML(learner, first_order=order, lr=inner_lr, allow_nograd=False)
learner.load_state_dict(torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt'))

<All keys matched successfully>

In [12]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-2.5233e-01, -1.3083e-02,  1.7343e-01],
                        [-6.0568e-02, -5.9060e-02,  1.0598e-01],
                        [-2.7154e-01, -1.7081e-02,  4.0474e-02]],
              
                       [[-3.5261e-01, -1.5088e-02,  3.7616e-02],
                        [-2.7602e-01,  4.7716e-02, -1.3594e-01],
                        [-5.7986e-02,  5.9228e-02, -1.3645e-01]],
              
                       [[-3.5785e-03,  4.1307e-02, -1.2321e-01],
                        [-1.9926e-01,  9.2488e-02,  7.2494e-02],
                        [-2.4462e-01,  4.5338e-02,  2.4103e-02]]],
              
              
                      [[[ 1.6439e-01, -3.5624e-02,  1.8537e-01],
                        [-1.6574e-01, -3.6141e-02,  2.3362e-01],
                        [ 1.2839e-01,  1.2165e-01,  8.9327e-02]],
              
                       [[ 7.7133e-02,  5.7750e-02,  2.0098e-01],
                        [ 1.04

In [7]:
_, _, test_tasks, learner = setup(
    args.dataset, args.root, args.n_ways, args.k_shots, args.q_shots, order, inner_lr, args.device, download=download, task_adapt=args.task_adapt, args=args)

learner.load_state_dict(torch.load('../logs/DELPO_tiered_5-way_1-shot_10-queries/exp5-tafe(wo relu)/model_67500_sd.pt'))
learner = learner.to(args.device)

FileNotFoundError: [Errno 2] No such file or directory: '../../dataset/mini_imagenet/tiered-imagenet/train_images_png.pkl'

In [11]:
learner.load_state_dict(torch.load('../logs/DELPO_miniimagenet_5-way_5-shot_10-queries/og/model_28500.pt'))
learner = learner.to(args.device)

In [10]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-1.9652e-02,  1.1479e-01,  1.9458e-01],
                        [ 7.3582e-03,  8.5265e-02,  1.7146e-01],
                        [ 5.4015e-02, -2.8553e-02,  4.8864e-02]],
              
                       [[-1.6846e-01,  2.0681e-01,  4.2507e-02],
                        [-8.0781e-02, -7.7319e-02,  1.4648e-01],
                        [ 4.1031e-02,  1.4342e-01, -4.7906e-02]],
              
                       [[-2.4925e-01, -3.5778e-02, -1.1827e-01],
                        [-3.2864e-01, -1.6313e-01, -1.0156e-01],
                        [-1.5666e-01, -1.1668e-02, -4.7175e-02]]],
              
              
                      [[[-1.2161e-01, -2.0437e-01, -2.7498e-01],
                        [ 7.3052e-02,  4.4535e-02,  6.3279e-02],
                        [ 1.8547e-01,  1.4081e-01,  3.4047e-02]],
              
                       [[ 1.8630e-02,  2.3010e-04, -4.1172e-01],
                        [-1.79

In [13]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 10
order = False
inner_lr = 0.00014
meta_lr = 0.000014
reconst_loss = nn.MSELoss(reduction='none')
adapt_steps = 10
meta_batch_size = 10
download = False

In [14]:
ttask = test_tasks.sample()

In [15]:
ttask

[tensor([[[[0.3579, 0.3519, 0.3442,  ..., 0.6730, 0.6768, 0.6775],
           [0.3559, 0.3529, 0.3489,  ..., 0.6771, 0.6784, 0.6944],
           [0.3628, 0.3504, 0.3346,  ..., 0.6761, 0.6771, 0.6881],
           ...,
           [0.3832, 0.3774, 0.3747,  ..., 0.6785, 0.6866, 0.6824],
           [0.3823, 0.3803, 0.3771,  ..., 0.6784, 0.6807, 0.6862],
           [0.3687, 0.3709, 0.6259,  ..., 0.6730, 0.6745, 0.6852]],
 
          [[0.4833, 0.4756, 0.4718,  ..., 0.8667, 0.8706, 0.8697],
           [0.4814, 0.4806, 0.4719,  ..., 0.8692, 0.8706, 0.8731],
           [0.4883, 0.4856, 0.4727,  ..., 0.8683, 0.8693, 0.8638],
           ...,
           [0.5636, 0.5578, 0.5528,  ..., 0.8589, 0.8627, 0.8745],
           [0.5589, 0.5607, 0.5551,  ..., 0.8510, 0.8611, 0.8666],
           [0.5588, 0.5502, 0.7682,  ..., 0.8534, 0.8549, 0.8578]],
 
          [[0.3364, 0.3098, 0.2874,  ..., 0.9961, 1.0000, 0.9962],
           [0.3248, 0.3155, 0.2884,  ..., 0.9987, 1.0000, 0.9990],
           [0.3275, 0.31

In [16]:
model = learner.clone()
evaluation_loss, evaluation_accuracy = inner_adapt_trident(
    ttask, reconst_loss, model, args.n_ways, args.k_shots, args.q_shots, adapt_steps, args.device, False, args)

/home/anuj/anaconda3/envs/torch/lib/python3.9/site-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/build/aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [17]:
evaluation_accuracy

tensor(0.6800)

In [18]:
evaluation_loss

{'elbo': tensor(426.0231, grad_fn=<AddBackward0>),
 'label_kl': tensor(15.8052, grad_fn=<MeanBackward0>),
 'style_kl': tensor(288.8022, grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(1779.1707, grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.0362, grad_fn=<NllLossBackward0>)}

: 